In [1]:
import pickle
import numpy as np
import pandas as pd
import re

seed = 2023

In [2]:
import os
os.chdir("../../")

In [3]:
from nlpsig_networks.scripts.ffn_baseline_functions import (
    histories_baseline_hyperparameter_search
)

/storage/ttseriotou/nlpsig-networks-env/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Rumours

In [4]:
%run notebooks/Rumours/load_sbert-embeddings.py

In [5]:
df_rumours.head()

,id,label,datetime,text,timeline_id,set
0,5.249902e+17,0,2014-10-22 18:26:23,Police have clarified that there were two shoo...,0,train
1,5.249906e+17,0,2014-10-22 18:27:58,"@CTVNews you guys ""confirmed"" there were 3 sho...",0,train
2,5.249908e+17,1,2014-10-22 18:28:46,@CTVNews get it right. http://t.co/GHYxMuzPG9,0,train
3,5.249927e+17,1,2014-10-22 18:36:29,RT @CTVNews Police have clarified that there w...,0,train
4,5.250038e+17,1,2014-10-22 19:20:41,@CTVNews @ctvsaskatoon so what happened at Rid...,0,train


In [6]:
sbert_embeddings.shape

torch.Size([5568, 384])

# Baseline: FFN using signatures

history_length = 20

In [7]:
dim_reduce_methods = ["umap"]
num_epochs = 100
batch=64
log_sig = True
log_signature_dimensions_and_sig_depths = [ (10, 3)]#[(28, 2), (10, 3), (6, 4)]
hidden_dim_sizes = [[32,32]]#[[32,32], [64,64], [128,128], [256,256]]
dropout_rates = [0.2]#[0.5, 0.2, 0.1]
learning_rates = [1e-4]#[1e-3, 1e-4, 5e-4]
seeds = [0, 1, 12, 123, 1234]
loss = "focal"
gamma = 2
validation_metric = "f1"

In [8]:
history_length = 20
ffn_logsignature_umap_20, best_ffn_logsignature_umap_20, _, __ = histories_baseline_hyperparameter_search(
    num_epochs=num_epochs,
    df=df_rumours,
    id_column="timeline_id",
    label_column="label",
    embeddings=sbert_embeddings.numpy(),
    y_data=y_data,
    output_dim=output_dim,
    hidden_dim_sizes=hidden_dim_sizes,
    dropout_rates=dropout_rates,
    learning_rates=learning_rates,
    use_signatures=True,
    seeds=seeds,
    loss=loss,
    gamma=gamma,
    batch_size=batch,
    log_signature=log_sig,
    dim_reduce_methods=dim_reduce_methods,
    dimension_and_sig_depths=log_signature_dimensions_and_sig_depths, 
    history_lengths=[history_length],
    path_indices=None,
    data_split_seed=123,
    split_ids= None, #torch.tensor(df_rumours['timeline_id'].astype(int)),
    split_indices = (df_rumours[df_rumours['set']=='train'].index, df_rumours[df_rumours['set']=='dev'].index, df_rumours[df_rumours['set']=='test'].index),
    k_fold=False,
    validation_metric=validation_metric,
    results_output=None,
    verbose=False
)

  0%|          | 0/1 [00:00<?, ?it/s]


[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' and feature...
[INFO] Adding 'time_diff' and feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.










































































































































































































































































































































100%|██████████| 5568/5568 [00:18<00:00, 308.22it/s]













100%|██████████| 1/1 [01:15<00:00, 75.13s/it]



100%|██████████| 1/1 [01:15<00:00, 75.13s/it]


100%|██████████| 1/1 [01:15<00:00, 75.14s/it]

100%|██████████| 1/1 [03:29<00:00, 209.28s/it]


[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' and feature...
[INFO] Adding 'time_diff' and feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


100%|██████████| 5568/5568 [00:17<00:00, 313.92it/s]


In [9]:
ffn_logsignature_umap_20

,loss,accuracy,f1,f1_scores,valid_loss,valid_accuracy,valid_f1,valid_f1_scores,hidden_dim,dropout_rate,...,seed,gamma,k_fold,model_id,k,input_dim,dimension,sig_depth,method,log_signature
0,focal,0.541468,0.533153,"[0.47084708470847086, 0.5954583683767872]",0.262089,0.601424,0.598735,"[0.631578947368421, 0.5658914728682171]","[32, 32]",0.2,...,0,2,False,0.0,20,1494,10,3,umap,True
0,focal,0.581506,0.568810,"[0.6427990235964198, 0.49482163406214036]",0.239177,0.594306,0.522451,"[0.7076923076923076, 0.3372093023255814]","[32, 32]",0.2,...,1,2,False,0.0,20,1494,10,3,umap,True
0,focal,0.593899,0.593150,"[0.5756972111553783, 0.6106032906764167]",0.247114,0.644128,0.625293,"[0.7093023255813954, 0.5412844036697249]","[32, 32]",0.2,...,12,2,False,0.0,20,1494,10,3,umap,True
0,focal,0.487131,0.487130,"[0.48664122137404575, 0.4876190476190476]",0.257492,0.580071,0.525881,"[0.6861702127659574, 0.3655913978494623]","[32, 32]",0.2,...,123,2,False,0.0,20,1494,10,3,umap,True
0,focal,0.589132,0.587399,"[0.6141450313339302, 0.5606523955147807]",0.305252,0.576512,0.456603,"[0.7118644067796611, 0.20134228187919465]","[32, 32]",0.2,...,1234,2,False,0.0,20,1494,10,3,umap,True


In [10]:
best_ffn_logsignature_umap_20["f1"].mean()

0.5539284308417417

In [11]:
np.stack(best_ffn_logsignature_umap_20["f1_scores"]).mean(axis=0)

array([0.55802591, 0.54983095])